# Roback Computational Problem
**Kyle Hood (JHU)**  
**Urban Economics, Fall 2025**  

In this problem, we will use Python to graphically display the canonical Roback graphs.

## Setup

For this problem, you should turn in one file of Python code. You have two choices for how to submit your answers to the questions in that file:
1. Embedded in the Python code by using the `print()` function
2. In comments of the program file.

For these problems, you should use the utility and cost functions from Brueckner’s problem 11.1. You can normalize consumers’ level of utility to 1 and the price of the export good to 1.

## Question I: Deriving Curves
1. Derive the curve mathematically (use your answers from Brueckner 11.1!)
    a. Write down the equation describing residents’ indifference curve (e.g., p as a function of y, a).
    b. Write down the equation describing firms’ isoprofit curve (e.g., p as a function of y, a, and θ).
2. Make a function:
    a. Called `p_resident_func` that takes y and a as inputs, and returns p
    b. Called `p_firm_func` that takes y, a, and theta as inputs, and returns p

## Question II: Indifference Points
1. Using the functions you generated
    a. Compute the land price at which an individual would obtain a utility of 1, if her income is 2 and there are 2 “units” of amenity.
    b. Compute the land price at which a firm earns zero profits, if wages are 2, and amenities don’t impact production.
2. Do the same calculation if wages (income) are 4.
3. Do the same calculation if wages (income) are 1.26.

## Question III: Plotting Equilibrium
For theta equals -0.1, 0, and 0.1, plot firm and household indifference curves for a = 1 and a = 1.5.
Each theta value should be on its own graph. Each of these should be labeled with the appropriate value of theta in the title. Also, each curve where a = 1 should be a solid line and each curve where a = 1.5 should be a dotted line.

Plot the firm curve in blue and the household curve in red. All of your graphs should range from 1 to 4 on the x axis.

To make this easier for you, here is the code necessary to do this for theta = 0.1:

```python
import numpy as np
import matplotlib.pyplot as plt

theta = 0.1
a = 1
y = np.arange(1, 4.01, 0.01)
plt.plot(y, p_firm_func(y, a, theta), color='blue', label='Firm, a=1')
plt.plot(y, p_resident_func(y, a), color='red', label='Resident, a=1')

a = 1.5
plt.plot(y, p_firm_func(y, a, theta), color='blue', linestyle='dotted', label='Firm, a=1.5')
plt.plot(y, p_resident_func(y, a), color='red', linestyle='dotted', label='Resident, a=1.5')

plt.title('theta = 0.1')
plt.xlabel('y')
plt.ylabel('p')
plt.legend()
plt.show()
```

To sum up, in your code you should have three plots, each of which should have 4 curves on it: firm and household curves for a = 1 and firm and household curves for a = 1.5.

## Question IV: Interpretation
For each of the cases graphed above, state whether equilibrium land prices and wages increase or decrease after a change in amenities from 1 to 1.5. Please explain your answers by reference to Brueckner chapter 11.2.

# Solution Question I.1

**Resident indifference curve**

The resident indifference curve is derived from the indirect utility function:

$$
\bar{u} = \frac{y}{2} \left( \frac{a}{p} \right)^{1/2}.
$$

Solving for $p$:

$$
\bar{u}^2 = \frac{y^2 a}{4p}
$$

$$
p = \frac{a y^2}{4 \bar{u}^2}.
$$

If $\bar{u} = 1$, this simplifies to:

$$
p = \frac{a y^2}{4}.
$$

---

**Firm zero-profit condition**

$$
p^{1/2} y^{1/2} a^{-\theta} = 1.
$$

Solving for $p$:

$$
p y a^{-2\theta} = 1
$$

$$
p = \frac{a^{2\theta}}{y}.
$$

# Question I: Deriving Curves
2. Make a function: 
    a. Called `p_resident_func` that takes y and a as inputs, and returns p. 
    b. Called `p_firm_func` that takes y, a, and theta as inputs, and returns p.

In [1]:
# Question I.2: Deriving Curves
from typing import Literal, Callable
import numpy as np

# Residents’ indifference curve-- part c
# p: price (of real estate), a: amenities, y: income, ubar: utility level
def p_resident_func(y: float | np.ndarray,
                    a: float | np.ndarray,
                    ubar: float = 1.0) -> float | np.ndarray:
    return (a * y**2) / (4.0 * ubar**2)

# Firms’ isoprofit curve-- part e
# p: price (of real estate), a: amenities, y: income (price of labor), theta: elasticity of output w.r.t. amenities
def p_firm_func(y: float | np.ndarray,
                a: float | np.ndarray,
                theta: float | np.ndarray) -> float | np.ndarray:
    return (a**(2.0 * theta)) / y

# Define a type for the economy, which can be either "firm" or "resident."
economy = Literal["firm", "resident"]
# Helper function to select the appropriate price function based on the economy type
def price_func(econ: economy) -> Callable:
    if econ == "firm":
        return p_firm_func
    elif econ == "resident":
        return p_resident_func
    else:
        raise ValueError("economy must be 'firm' or 'resident'")

In [2]:
import numpy as np
import pandas as pd

def generate_data(
    economy_side: economy,
    *,
    a_series: np.ndarray | None = None,
    theta_series: np.ndarray | None = None,
    y_series: np.ndarray | None = None,
    ubar: float = 1.0,
) -> pd.DataFrame:
    
    pfunc = price_func(economy_side)   # <- using your selector name

    if a_series is None:
        a_series = np.linspace(0.1, 10.0, 60)
    if y_series is None:
        y_series = np.linspace(1.0, 100.0, 60)

    if economy_side == "firm":
        if theta_series is None:
            theta_series = np.linspace(-1.0, 1.0, 60)

        # 3D grid
        A = a_series[None, None, :]        # (1, 1, na)
        T = theta_series[None, :, None]    # (1, nt, 1)
        Y = y_series[:, None, None]        # (ny, 1, 1)

        P = pfunc(y=Y, a=A, theta=T)       # (ny, nt, na)

        df = pd.DataFrame({
            "a": np.broadcast_to(A, P.shape).ravel(),
            "y": np.broadcast_to(Y, P.shape).ravel(),
            "theta": np.broadcast_to(T, P.shape).ravel(),
            "p_firm": P.ravel(),
        })
        return df

    elif economy_side == "resident":
        # 2D grid (no theta)
        A = a_series[None, :]              # (1, na)
        Y = y_series[:, None]              # (ny, 1)

        P = pfunc(y=Y, a=A, ubar=ubar)     # (ny, na)

        df = pd.DataFrame({
            "a": np.broadcast_to(A, P.shape).ravel(),
            "y": np.broadcast_to(Y, P.shape).ravel(),
            "u_bar": ubar,
            "p_resident": P.ravel(),
        })
        return df

    else:
        raise ValueError("economy_side must be 'firm' or 'resident'")

#-----------------------------
df_firm = generate_data("firm")
df_resident = generate_data("resident")

df_firm.head(), df_firm.shape, '-'*35, df_resident.head(), df_resident.shape


(          a    y  theta      p_firm
 0  0.100000  1.0   -1.0  100.000000
 1  0.267797  1.0   -1.0   13.944079
 2  0.435593  1.0   -1.0    5.270330
 3  0.603390  1.0   -1.0    2.746654
 4  0.771186  1.0   -1.0    1.681439,
 (216000, 4),
 '-----------------------------------',
           a    y  u_bar  p_resident
 0  0.100000  1.0    1.0    0.025000
 1  0.267797  1.0    1.0    0.066949
 2  0.435593  1.0    1.0    0.108898
 3  0.603390  1.0    1.0    0.150847
 4  0.771186  1.0    1.0    0.192797,
 (3600, 4))

In [3]:
import numpy as np
import pandas as pd

# Ensure sorted unique grids taken from your actual data
A_vals = np.sort(df_firm["a"].unique())
Y_vals = np.sort(df_firm["y"].unique())
T_vals = np.sort(df_firm["theta"].unique())

# Pivot resident once: (y x a) -> p_resident
res_pivot = df_resident.pivot(index="y", columns="a", values="p_resident").sort_index()

# For firm, we’ll build a dict of pivots by theta for quick slicing
firm_pivots = {}
for t in T_vals:
    sub = df_firm[df_firm["theta"] == t]
    firm_pivots[t] = sub.pivot(index="y", columns="a", values="p_firm").sort_index()


In [ ]:
import matplotlib.pyplot as plt
def roback_shift_plot(df_firm, df_resident, *, theta_val, a1, a2):
    firm1 = df_firm[np.isclose(df_firm["a"], a1) & np.isclose(df_firm["theta"], theta_val)].sort_values("y")
    firm2 = df_firm[np.isclose(df_firm["a"], a2) & np.isclose(df_firm["theta"], theta_val)].sort_values("y")
    res1  = df_resident[np.isclose(df_resident["a"], a1)].sort_values("y")
    res2  = df_resident[np.isclose(df_resident["a"], a2)].sort_values("y")

    plt.figure(figsize=(7,4))
    plt.plot(firm1["y"], firm1["p_firm"], color="blue", linestyle="-", label=f"Firm a={a1:.3g}")
    plt.plot(firm2["y"], firm2["p_firm"], color="blue", linestyle=":", label=f"Firm a={a2:.3g}")
    plt.plot(res1["y"], res1["p_resident"], color="red", linestyle="-", label=f"Resident a={a1:.3g}")
    plt.plot(res2["y"], res2["p_resident"], color="red", linestyle=":", label=f"Resident a={a2:.3g}")

    plt.xlabel("Wage (y)")
    plt.ylabel("Land price (p)")
    plt.title(f"Amenity shift at θ={theta_val:.3g}")
    plt.legend()
    plt.show()

theta_val = df_firm["theta"].unique()[30]
a1 = df_firm["a"].unique()[5]
a2 = df_firm["a"].unique()[45]

roback_shift_plot(df_firm, df_resident, theta_val=theta_val, a1=a1, a2=a2)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def roback_plot(df_firm, df_resident, *, a_val, theta_val, ubar=1.0):
    # firm slice (use isclose to avoid float equality issues)
    firm = df_firm[np.isclose(df_firm["a"], a_val) & np.isclose(df_firm["theta"], theta_val)].sort_values("y")
    res  = df_resident[np.isclose(df_resident["a"], a_val)].sort_values("y")

    plt.figure(figsize=(7,4))
    plt.plot(firm["y"], firm["p_firm"], label="Firm", color="blue")
    plt.plot(res["y"], res["p_resident"], label="Resident", color="red")
    plt.xlabel("Wage (y)")
    plt.ylabel("Land price (p)")
    plt.title(f"Roback curves at a={a_val:.3g}, θ={theta_val:.3g}")
    plt.legend()
    plt.show()

# pick values that actually exist in your grids:
a_val = df_firm["a"].unique()[10]
theta_val = df_firm["theta"].unique()[30]

roback_plot(df_firm, df_resident, a_val=a_val, theta_val=theta_val)

In [4]:
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def _closest_intersection(y_grid, p1, p2):
    # returns y*, p* at point minimizing |p1 - p2|
    diff = np.abs(p1 - p2)
    idx = int(np.nanargmin(diff))
    return y_grid[idx], p1[idx]

def plot_roback(theta, a1, a2):
    # grab curves from pivots (no float comparisons, uses exact columns)
    firm_pivot = firm_pivots[theta]

    y = firm_pivot.index.to_numpy()

    p_firm_1 = firm_pivot[a1].to_numpy()
    p_res_1  = res_pivot[a1].to_numpy()

    p_firm_2 = firm_pivot[a2].to_numpy()
    p_res_2  = res_pivot[a2].to_numpy()

    # intersections
    y1_star, p1_star = _closest_intersection(y, p_firm_1, p_res_1)
    y2_star, p2_star = _closest_intersection(y, p_firm_2, p_res_2)

    fig, ax = plt.subplots(1, 2, figsize=(12, 4), sharey=True)

    # --- Left: a1 ---
    ax[0].plot(y, p_firm_1, label="Firm", color="blue")
    ax[0].plot(y, p_res_1, label="Resident", color="red")
    ax[0].scatter([y1_star], [p1_star], color="black", s=30)
    ax[0].set_title(f"a = {a1:.3g}, θ = {theta:.3g}")
    ax[0].set_xlabel("Wage (y)")
    ax[0].set_ylabel("Land price (p)")
    ax[0].legend()

    # --- Right: a2 ---
    ax[1].plot(y, p_firm_2, label="Firm", color="blue")
    ax[1].plot(y, p_res_2, label="Resident", color="red")
    ax[1].scatter([y2_star], [p2_star], color="black", s=30)
    ax[1].set_title(f"a = {a2:.3g}, θ = {theta:.3g}")
    ax[1].set_xlabel("Wage (y)")
    ax[1].legend()

    plt.tight_layout()
    plt.show()

# Sliders that snap to your grid values
theta_slider = FloatSlider(value=float(T_vals[len(T_vals)//2]),
                           min=float(T_vals.min()),
                           max=float(T_vals.max()),
                           step=float(np.diff(T_vals).min()),
                           description="theta")

a1_slider = FloatSlider(value=float(A_vals[0]),
                        min=float(A_vals.min()),
                        max=float(A_vals.max()),
                        step=float(np.diff(A_vals).min()),
                        description="a1")

a2_slider = FloatSlider(value=float(A_vals[-1]),
                        min=float(A_vals.min()),
                        max=float(A_vals.max()),
                        step=float(np.diff(A_vals).min()),
                        description="a2")

interact(plot_roback, theta=theta_slider, a1=a1_slider, a2=a2_slider);


interactive(children=(FloatSlider(value=0.016949152542372836, description='theta', max=1.0, min=-1.0, step=0.0…

## Question II: Indifference Points
1. Using the functions you generated
   
    a. Compute the land price at which an individual would obtain a utility of 1, if her income is 2 and there are 2 “units” of amenity.
    
    b. Compute the land price at which a firm earns zero profits, if wages are 2, and amenities don’t impact production.
2. Do the same calculation if wages (income) are 4.
3. Do the same calculation if wages (income) are 1.26.

In [ ]:
# Question II: Indifference Points
# Set a and theta
a = 2.0      # amenities
ubar = 1.0   # utility level
theta = 0.0  # elasticity of output w.r.t. amenities

# compute land price for y=2 for individual utility = 1 and profits = 0
#y=2, a=2, ubar=1, theta=0
land_resident_price_2_2_1 = p_resident_func(y=2, a=a, ubar=ubar)
land_firm_price_2_2_0     = p_firm_func(y=2, a=2, theta=0)

# compute land price for y=4 for individual utility = 1 and profits = 0
#y=4, a=2, ubar=1, theta=0
land_resident_price_4_2_1 = p_resident_func(y=4, a=a, ubar=ubar)
land_firm_price_4_2_0     = p_firm_func(y=4, a=2, theta=0)

# compute land price for y=1.26 for individual utility = 1 and profits = 0
#y=1.26, a=2, ubar=1, theta=0
land_resident_price_1_26_2_1 = p_resident_func(y=1.26, a=a, ubar=ubar)
land_firm_price_1_26_2_0     = p_firm_func(y=1.26, a=a, theta=theta)

## Question III: Plotting Equilibrium
For theta equals -0.1, 0, and 0.1, plot firm and household indifference curves for a = 1 and a = 1.5.
Each theta value should be on its own graph. Each of these should be labeled with the appropriate value of theta in the title. Also, each curve where a = 1 should be a solid line and each curve where a = 1.5 should be a dotted line.

Plot the firm curve in blue and the household curve in red. All of your graphs should range from 1 to 4 on the x axis.

In [ ]:
# Question III: Plotting Equilibrium
# import necessary libraries
#your code here

# set parameters and range for a
#your code here

# Plotting the curves for different values of theta and a
#your code

# Question IV: Interpretation
For each of the cases graphed above, state whether equilibrium land prices and wages increase or decrease after a change in amenities from 1 to 1.5. Please explain your answers by reference to Brueckner chapter 11.2.?

[your answer here]